## Learning Lyapunov function for Inverted Pendulum

In [1]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt

## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 2×2 identity matrix and R is 1×1 identity matrix

In [2]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1,bias=False)
        self.control.weight = torch.nn.Parameter(lqr)

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = self.control(x)
        return out,u

## Dynamical system

In [3]:
def f_value(x,u):
    #Dynamics
    y = []
    G = 9.81  # gravity
    L = 0.5   # length of the pole 
    m = 0.15  # ball mass
    b = 0.1   # friction
    
    for r in range(0,len(x)): 
        f = [ x[r][1], 
              (m*G*L*np.sin(x[r][0])- b*x[r][1]) / (m*L**2)]
        y.append(f) 
    y = torch.tensor(y)
    y[:,1] = y[:,1] + (u[:,0]/(m*L**2))
    return y

## Options

In [4]:
'''
For learning 
'''
N = 500             # sample size
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)  

x = torch.Tensor(N, D_in).uniform_(-6, 6)           
x_0 = torch.zeros([1, 2])

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
vars_ = [x1,x2]
G = 9.81 
l = 0.5  
m = 0.15
b = 0.1
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
epsilon = 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.5
ball_ub = 6

## Learning and Falsification

In [5]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    lqr = torch.tensor([[-23.58639732,  -5.31421063]])    # lqr solution
    model = Net(D_in,H1, D_out,lqr)
    L = []
    i = 0 
    t = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        V_candidate, u = model(x)
        X0,u0 = model(x_0)
        f = f_value(x,u)
        Circle_Tuning = Tune(x)
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

        # With tuning
        Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()\
                    +2.2*((Circle_Tuning-6*V_candidate).pow(2)).mean()


        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        w1 = model.layer1.weight.data.numpy()
        w2 = model.layer2.weight.data.numpy()
        b1 = model.layer1.bias.data.numpy()
        b2 = model.layer2.bias.data.numpy()
        q = model.control.weight.data.numpy()

        # Falsification
        if i % 10 == 0:
            u_NN = (q.item(0)*x1 + q.item(1)*x2) 
            f = [ x2,
                 (m*G*l*sin(x1) + u_NN - b*x2) /(m*l**2)]

            # Candidate V
            z1 = np.dot(vars_,w1.T)+b1

            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            V_learn = tanh(z2.item(0))

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            result= CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config,epsilon)
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = AddCounterexamples(x,result,10)
            else:  
                valid = True
                print("Satisfy conditions!!")
                print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        i += 1

    stop = timeit.default_timer()


    np.savetxt("w1.txt", model.layer1.weight.data, fmt="%s")
    np.savetxt("w2.txt", model.layer2.weight.data, fmt="%s")
    np.savetxt("b1.txt", model.layer1.bias.data, fmt="%s")
    np.savetxt("b2.txt", model.layer2.bias.data, fmt="%s")
    np.savetxt("q.txt", model.control.weight.data, fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

(0, 'Lyapunov Risk=', 73.07493591308594)
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-5.607981400644867165, -5.607370072610038392]
x2 : [0.9742785792574937265, 0.974595727232512421]
(1, 'Lyapunov Risk=', 60.24591827392578)
(2, 'Lyapunov Risk=', 53.66139602661133)
(3, 'Lyapunov Risk=', 49.358272552490234)
(4, 'Lyapunov Risk=', 44.88718032836914)
(5, 'Lyapunov Risk=', 40.768131256103516)
(6, 'Lyapunov Risk=', 37.115272521972656)
(7, 'Lyapunov Risk=', 33.89949417114258)
(8, 'Lyapunov Risk=', 31.116905212402344)
(9, 'Lyapunov Risk=', 28.676088333129883)
(10, 'Lyapunov Risk=', 26.526872634887695)
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.2044197886113856, -0.1944197886113855911]
x2 : [0.5114223581404099273, 0.5214223581404099361]
(11, 'Lyapunov Risk=', 24.382814407348633)
(12, 'Lyapunov Risk=', 23.120990753173828)
(13, 'Lyapunov Risk=', 21.971805572509766)
(14, 'Lyapunov Risk=', 20.570842742919922)
(15, '

(133, 'Lyapunov Risk=', 6.618300437927246)
(134, 'Lyapunov Risk=', 6.6116623878479)
(135, 'Lyapunov Risk=', 6.604804515838623)
(136, 'Lyapunov Risk=', 6.597716808319092)
(137, 'Lyapunov Risk=', 6.590392112731934)
(138, 'Lyapunov Risk=', 6.582840919494629)
(139, 'Lyapunov Risk=', 6.575015068054199)
(140, 'Lyapunov Risk=', 6.566971302032471)
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.1097123389205162391, -0.09971233892051623027]
x2 : [0.5217048507572324967, 0.5317048507572325056]
(141, 'Lyapunov Risk=', 6.924576282501221)
(142, 'Lyapunov Risk=', 6.915141582489014)
(143, 'Lyapunov Risk=', 6.905326843261719)
(144, 'Lyapunov Risk=', 6.89514684677124)
(145, 'Lyapunov Risk=', 6.884635925292969)
(146, 'Lyapunov Risk=', 6.873801231384277)
(147, 'Lyapunov Risk=', 6.862657070159912)
(148, 'Lyapunov Risk=', 6.8512163162231445)
(149, 'Lyapunov Risk=', 6.839859485626221)
(150, 'Lyapunov Risk=', 6.828097820281982)
===========Verifying==========
Not a Lyapu

(264, 'Lyapunov Risk=', 5.946793556213379)
(265, 'Lyapunov Risk=', 5.916933059692383)
(266, 'Lyapunov Risk=', 5.886115074157715)
(267, 'Lyapunov Risk=', 5.854376316070557)
(268, 'Lyapunov Risk=', 5.821198463439941)
(269, 'Lyapunov Risk=', 5.7871270179748535)
(270, 'Lyapunov Risk=', 5.751532077789307)
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-0.1565876609556311472, -0.1494270066630787597]
x2 : [0.7321620494688090286, 0.7410810247344046253]
(271, 'Lyapunov Risk=', 5.761460304260254)
(272, 'Lyapunov Risk=', 5.723428249359131)
(273, 'Lyapunov Risk=', 5.683546543121338)
(274, 'Lyapunov Risk=', 5.645268440246582)
(275, 'Lyapunov Risk=', 5.605838775634766)
(276, 'Lyapunov Risk=', 5.5663957595825195)
(277, 'Lyapunov Risk=', 5.527245044708252)
(278, 'Lyapunov Risk=', 5.489575386047363)
(279, 'Lyapunov Risk=', 5.458305358886719)
(280, 'Lyapunov Risk=', 5.429406642913818)
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 

(392, 'Lyapunov Risk=', 3.822031259536743)
(393, 'Lyapunov Risk=', 3.8126914501190186)
(394, 'Lyapunov Risk=', 3.806110143661499)
(395, 'Lyapunov Risk=', 3.798611879348755)
(396, 'Lyapunov Risk=', 3.798466444015503)
(397, 'Lyapunov Risk=', 3.7911112308502197)
(398, 'Lyapunov Risk=', 3.7854206562042236)
(399, 'Lyapunov Risk=', 3.7765698432922363)
(400, 'Lyapunov Risk=', 3.7708206176757812)
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-2.063964843750000444, -2.062500000000000444]
x2 : [5.621731063570308606, 5.623536225987946224]
(401, 'Lyapunov Risk=', 3.756504535675049)
(402, 'Lyapunov Risk=', 3.7504842281341553)
(403, 'Lyapunov Risk=', 3.7439329624176025)
(404, 'Lyapunov Risk=', 3.7383737564086914)
(405, 'Lyapunov Risk=', 3.7296383380889893)
(406, 'Lyapunov Risk=', 3.72628116607666)
(407, 'Lyapunov Risk=', 3.717179775238037)
(408, 'Lyapunov Risk=', 3.7117226123809814)
(409, 'Lyapunov Risk=', 3.7057888507843018)
(410, 'Lyapunov Risk=', 3.698866128

### Checking result with smaller epsilon

In [6]:
epsilon = -0.00001
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, epsilon)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
else:  
    print("Satisfy conditions with epsilon= ",epsilon)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

('Satisfy conditions with epsilon= ', -1e-05)
(<Expression "tanh((0.47052764892578125 - 0.36160281300544739 * tanh((-0.75011157989501953 + 0.0088593903928995132 * x1 - 0.0036623133346438408 * x2)) + 0.79846572875976562 * tanh((-0.6316143274307251 + 0.73739403486251831 * x1 - 0.0083910385146737099 * x2)) + 0.13357226550579071 * tanh((-0.57977020740509033 - 0.31221801042556763 * x1 - 0.1601211279630661 * x2)) - 0.94644665718078613 * tanh((0.81671983003616333 + 0.92968451976776123 * x1 - 0.01118092518299818 * x2)) - 0.16966792941093445 * tanh((1.0393475294113159 - 0.6808173656463623 * x1 - 0.21255317330360413 * x2)) + 0.93776065111160278 * tanh((1.1187564134597778 + 0.00042254151776432991 * x1 - 8.0787671322468668e-05 * x2))))">, ' is a Lyapunov function.')
